In [52]:
import numpy
import random
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Conv1D, Activation, MaxPooling1D, Flatten, Dense
from hyperopt import hp, STATUS_OK, fmin, tpe, Trials, space_eval

In [50]:
xs, ys = numpy.load('xs.npy'), numpy.load('ys.npy')
i_s = random.shuffle(range(len(xs)))
xs, ys = [xs[i] for i in i_s], [ys[i] for i in i_s]

In [3]:
def objective(ps):
    x = Input(shape=xs[0].shape)
    h = MaxPooling1D()(Activation(ps['func'])(Conv1D(filters=ps['size'], kernel_size=ps['conv'])(x)))
    h = MaxPooling1D()(Activation(ps['func'])(Conv1D(filters=ps['size'], kernel_size=ps['conv'])(h)))
    y = Dense(units=len(ys[0]))(Flatten()(h))
    model = Model(inputs=x, outputs=y)
    model.compile(loss='mae', optimizer=Adam(amsgrad=ps['amsg']))
    history = model.fit(x=xs, y=ys, validation_split=0.0625, batch_size=2, epochs=64, verbose=0)
    return { 'loss': numpy.mean(sorted(history.history['val_loss'])[:4]), 'status': STATUS_OK }

In [4]:
space = {
    'size': hp.choice('size', [16, 32, 64]),
    'conv': hp.choice('conv', [3, 5, 7]),
    'func': hp.choice('func', ['relu', 'elu']),
    'amsg': hp.choice('amsg', [True, False])
}

In [5]:
trials = Trials()
best = fmin(fn=objective, space=space, algo=tpe.suggest, max_evals=64, trials=trials)
space_eval(space, best)

{'amsg': True, 'conv': 5, 'func': 'elu', 'size': 64}

In [65]:
x = Input(shape=xt[0].shape)
h = MaxPooling1D()(Activation('elu')(Conv1D(filters=64, kernel_size=5)(x)))
h = MaxPooling1D()(Activation('elu')(Conv1D(filters=64, kernel_size=5)(h)))
y = Dense(units=len(yt[0]))(Flatten()(h))
model = Model(inputs=x, outputs=y)
model.compile(loss='mae', optimizer=SGD(lr=0.001, momentum=0.95))
model.fit(x=xt, y=yt, validation_data=(xv, yv), batch_size=2, epochs=64, verbose=2)

Train on 927 samples, validate on 62 samples
Epoch 1/64
 - 12s - loss: 0.0741 - val_loss: 0.0355
Epoch 2/64
 - 5s - loss: 0.0348 - val_loss: 0.0438
Epoch 3/64
 - 5s - loss: 0.0299 - val_loss: 0.0261
Epoch 4/64
 - 4s - loss: 0.0292 - val_loss: 0.0233
Epoch 5/64
 - 4s - loss: 0.0268 - val_loss: 0.0201
Epoch 6/64
 - 4s - loss: 0.0257 - val_loss: 0.0226
Epoch 7/64
 - 4s - loss: 0.0232 - val_loss: 0.0157
Epoch 8/64
 - 4s - loss: 0.0218 - val_loss: 0.0214
Epoch 9/64


KeyboardInterrupt: 